In [1]:
import pandas as pd
import numpy as np
import scipy as sp
import plotly.plotly as py
import plotly.figure_factory as ff
import plotly.graph_objs as go
import scipy.signal as ss
from statistics import median, mean

In [2]:
from selfpkg import readDB, MinToMS
from dataFilter import butter_lowpass_filter

In [3]:
#set credentials file for plotly(username & API key)
import plotly
username = 'mengyizhou95' #username registered on Ploty
api_key = 'X5DOiJATuUqPbsabrbxA' #the api key plotly asigned
plotly.tools.set_credentials_file(username=username, api_key=api_key)

In [4]:
#set the conversion ratio from min to ms
timeConversionVal = 60000

order = 5 #order of filter (1-)
fs = 25       # sample rate, Hz
cutoff = 10 # desired cutoff frequency of the filter, Hz

In [21]:
def edgeDetection(df, dfOrientation, start=None, end=None, filename='edgeDetection'):
    
    if start is None:
        start = df.iloc[0]['time'].item()
    else:
        start = MinToMS(start, df)
    if end is None:
        end = df.tail(1)['time'].item()
    else:
        end = MinToMS(end, df)
    
    startIndex = df.index[df['time'] == start].tolist()[0]
    endIndex = df.index[df['time'] == end].tolist()[0]
    
    sigma = 50
    dfOrientationProcessed = sp.ndimage.filters.gaussian_filter1d(dfOrientation[startIndex: endIndex], sigma)
#     dfOrientationProcessed = dfOrientation[startIndex: endIndex]
    dfTime = df.time[startIndex: endIndex].values.tolist()

    mask = [-1, -1/2, 0, 1/2, 1] 
    orientCon = np.convolve(dfOrientationProcessed, mask)
    
    trace0 = go.Scattergl(
        x = [x / timeConversionVal for x in dfTime],
        y = orientCon,
        line = dict(
            color = ('rgb(60, 60, 225)'),
            width = 1
        )
    )
    
    trace1 = go.Scattergl(
        x = [x / timeConversionVal for x in dfTime],
        y = butter_lowpass_filter(orientCon, cutoff, fs, order),
        line = dict(
            color = ('rgb(225, 60, 60)'),
            width = 1
        )
    )
    
    data = [trace0, trace1]
    layout = dict(title = filename,
                  xaxis = dict(title = 'time'),
                  yaxis = dict(title = 'orientation convolution '))
    fig = dict(data = data, layout = layout)
    #plotly.offline.plot(fig, filename=filename + '.html')
    return orientCon, dfTime

In [5]:
def rangeSelectionOneDirection(orientCon, dfTime):    
    i = 0
    j = 0
    rangeList = []
    #select first range by 0
    while(j < len(orientCon)):
        if i  == j:
            if orientCon[j] == 0:
                i += 1
                j += 1
            else:
                j += 1
        else:
            if orientCon[j] == 0:
                rangeList.append(i)
                rangeList.append(j)
                i = j
            else:
                j += 1
    #merge close range
    i = 0            
    while(i < len(rangeList)):
        if i == len(rangeList) - 1:
            break
        if i % 2 == 0:
            i += 1
        else:
            if rangeList[i+1] - rangeList[i] < 2:
                del rangeList[i]
                del rangeList[i]
            else:
                i += 1  
    #delete the almost zero
    i = 0
    while(i < len(rangeList)-1):
        if i % 2 != 0:
            i += 1
        else:
            if median(map(abs, orientCon[rangeList[i]: rangeList[i+1]])) < 0.01:
                del rangeList[i]
                del rangeList[i]
            else:
                i += 1

    rangeList.insert(0, 0)
    rangeList.append(len(orientCon)-1)

    i = 1
    flag = 0
    s = 0
    e = 0
    #the range should have 0 @front & end
    while(i < len(rangeList)-2):
        if rangeList[i] - rangeList[i-1] > 3000:
            flag += 1
        if rangeList[i+2] - rangeList[i+1] > 3000:
            flag += 1

        if flag == 2:
            s = rangeList[i]
            e = rangeList[i+1]
            break
        else:
            flag = 0
            i += 2
    return dfTime[s], dfTime[e]

In [6]:
def rangeSelection4orientation(df):
    dfOrientationX = df.orientationX
    dfOrientationY = df.orientationY
    orientConX, timeX = edgeDetection(df, dfOrientationX)
    orientConY, timeY = edgeDetection(df, dfOrientationY)
    startX, endX = rangeSelectionOneDirection(orientConX, timeX)
    startY, endY = rangeSelectionOneDirection(orientConX, timeX)
    start = max(startX, startY)
    end = min(endX, endY)
    return start, end

In [19]:
csv_database = 'sqlite:///Cambridge2.db'
df = readDB(csv_database)
start, end = rangeSelection4orientation(df)

In [20]:
print(start/timeConversionVal)
print(end/timeConversionVal)

9.583833333333333
16.89281666666667


In [7]:
def edgeDetection4compass(df, start=None, end=None, filename='edgeDetection'):
    
    if start is None:
        start = df.iloc[0]['time'].item()
    else:
        start = MinToMS(start, df)
    if end is None:
        end = df.tail(1)['time'].item()
    else:
        end = MinToMS(end, df)
    
    startIndex = df.index[df['time'] == start].tolist()[0]
    endIndex = df.index[df['time'] == end].tolist()[0]
    
    x = df.compassX
    y = df.compassY
    z = df.compassZ
    v = (((x ** 2 + y ** 2) ** (1/2)) ** 2 + z ** 2) ** (1/2)
    
    sigma = 50
    dfOrientationProcessed = sp.ndimage.filters.gaussian_filter1d(v[startIndex: endIndex], sigma)

    dfTime = df.time[startIndex: endIndex].values.tolist()

    mask = [-1, -1/2, 0, 1/2, 1] 
    orientCon = np.convolve(dfOrientationProcessed, mask)
    
#     trace0 = go.Scattergl(
#         x = [x / timeConversionVal for x in dfTime],
#         y = orientCon,
#         line = dict(
#             color = ('rgb(60, 60, 225)'),
#             width = 1
#         )
#     )
    
#     trace1 = go.Scattergl(
#         x = [x / timeConversionVal for x in dfTime],
#         y = butter_lowpass_filter(orientCon, cutoff, fs, order),
#         line = dict(
#             color = ('rgb(225, 60, 60)'),
#             width = 1
#         )
#     )
    
#     data = [trace0]#, trace1]
#     layout = dict(title = filename,
#                   xaxis = dict(title = 'time'),
#                   yaxis = dict(title = 'orientation convolution '))
#     fig = dict(data = data, layout = layout)
#     plotly.offline.plot(fig, filename=filename + '.html')
    return orientCon, dfTime

In [15]:
csv_database = 'sqlite:///Cambridge2.db'
df = readDB(csv_database)
compassCon, time = edgeDetection4compass(df, filename = 'compassProcessed')

In [16]:
startC, endC = rangeSelectionOneDirection(compassCon, time)
print(startC/timeConversionVal)
print(endC/timeConversionVal)

0.10036666666666667
0.10036666666666667
